In [1]:
import pandas as pd
from sqlalchemy import create_engine
from time import time

In [2]:
df = pd.read_csv('green_tripdata_2019-10.csv', nrows=100)

In [18]:
user = 'postgres'
password = 'root'
host = 'localhost'
port = '5433'
database = 'ny_taxi'

In [19]:
connection_str = f"postgresql://{user}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_str)

In [20]:
try:
    with engine.connect() as connection_str:
        print('Successfully connected to the PostgreSQL database')
except Exception as ex:
    print(f'Sorry failed to connect: {ex}')

Successfully connected to the PostgreSQL database


In [4]:
print(df.io.sql.get_schema(df, name='yellow_taxi_data'))

AttributeError: 'DataFrame' object has no attribute 'io'

In [21]:
df_iter = pd.read_csv('green_tripdata_2019-10.csv', iterator=True, chunksize=100000)

In [22]:
df = next(df_iter)
len(df)

100000

In [23]:
df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)

In [24]:
df.head(n=1)

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2019-10-01 00:26:02,2019-10-01 00:39:58,N,1,112,196,1,5.88,18.0,0.5,0.5,0.0,0.0,NaN,0.3,19.3,2,1.0,0.0


In [26]:
df.to_sql(name='yellow_taxi_data', con=engine, if_exists='replace')

1000

In [27]:
%time df.to_sql(name='yellow_taxi_data', con=engine, if_exists='replace')

CPU times: total: 4.42 s
Wall time: 9.17 s


1000

In [ ]:
while True:
    t_start = time()
    df = next(df_iter)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.to_sql(name='yellow_taxi_data', con=engine, if_exists='replace')
    t_end = time()
    print("inserted another chunk data...., took %.3f" % (t_end - t_start))